In [18]:
import pandas as pd
import datetime
import numpy as np
import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
epoch = datetime.datetime.utcfromtimestamp(0)

In [19]:
def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [2]:
df = pd.read_csv('data_train_partial.csv')

In [9]:
X = df.iloc[:,[0,1,2,3]]
y = df.iloc[:,[4]].values

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [42]:
import xgboost as xgb
# specify parameters via map
classifier = xgb.XGBClassifier(eta=0.2, max_depth=6, n_estimators=1000)
classifier.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.2, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [43]:
y_predicted= classifier.predict(X_test)
accuracy_score(y_test,y_predicted)

0.9262208268714301

In [52]:
df_test = pd.read_csv('data_test.csv', parse_dates=['time_exit', 'time_entry'])
df_test = df_test.drop_duplicates(subset='hash', keep='last')

In [53]:
len(df_test)

33515

In [54]:
df_test_new_entry = df_test['time_entry'].apply(unix_time_millis)
df_test_new_exit = df_test['time_exit'].apply(unix_time_millis)

In [55]:
df_test['time_entry'] = df_test_new_entry
df_test['time_exit'] = df_test_new_exit

In [56]:
df_test.head()

,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
4,4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,1.554822e+12,1.554823e+12,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN
7,7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,1.554824e+12,1.554824e+12,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN
10,10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,1.554824e+12,1.554824e+12,NaN,NaN,NaN,3.757468e+06,-1.923860e+07,NaN,NaN
13,13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,1.554824e+12,1.554825e+12,NaN,NaN,NaN,3.760505e+06,-1.935500e+07,NaN,NaN
17,17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,1.554822e+12,1.554822e+12,NaN,NaN,NaN,3.766319e+06,-1.917013e+07,NaN,NaN


In [58]:
#df_test.to_csv('data_test_clean.csv', index=False)

In [59]:
X_dt_test = df_test.iloc[:, [3,4,8,9]]

In [46]:
y_pred = classifier.predict(X_dt_test)

In [47]:
df_final = df_test.iloc[:, [2]]

In [48]:
df_final['target'] = y_pred

/userhome/comp3314/c3314g15/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
df_final.columns=['id','target']

In [50]:
df_final.head()

,id,target
4,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
7,traj_000479418b5561ab694a2870cc04fd43_25_10,0
10,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
13,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
17,traj_00063a4f6c12e1e4de7d876580620667_3_4,0


In [51]:
df_final.to_csv('result_XGBoost.csv', index=False)